## Loading library

In [7]:
library(edgeR)
library(ggplot2)
library("ggrepel")

## Define filter

In [102]:
column_names = list('Time.point' = c("4mo"), #c("12mo"),
                    'Sex' = c("F"), #c("F", "M"),
                    'Group' = c("5xFADWT:Bin1HO", "5xFADWT")) #c("5xFADHEMI:Bin1HO", "5xFADHEMI"))

## Read the data
you should have to prepare two data:
1. count matrix on polyA genes
2. metadata (with the same order of data set 1)


In [103]:
#read table (RNA_seq)
datExpr = read.csv("data/countMatrix_sorted_polyA.csv", row.name=1, sep=",", header = T, check.names = FALSE)

for each comparison, you need to repeat this part

## Subset data

if you want to only consider subset of samples instead of whole you should remove those samples from count matrix
In this part of the code, you will keep the information about those samples that you care about it!

In [104]:
datTraits = read.csv("data/experimentList_sorted.csv", row.name=1, sep=",", header = T)
datTraits$keep = TRUE
head(datTraits)

,file.name,Time.point,Tissue,Sex,Group,keep
,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
0,hipp_F_5xFADHEMI_4mo_B1_11616_S17_rsem.genes.results,4mo,hipp,F,5xFADHEMI,TRUE
1,hipp_F_5xFADHEMI_4mo_B1_11626_S19_rsem.genes.results,4mo,hipp,F,5xFADHEMI,TRUE
2,hipp_F_5xFADHEMI_4mo_B2_11617_S38_rsem.genes.results,4mo,hipp,F,5xFADHEMI,TRUE
3,hipp_F_5xFADHEMI_4mo_B1_11625_S18_rsem.genes.results,4mo,hipp,F,5xFADHEMI,TRUE
4,hipp_F_5xFADHEMI_4mo_B1_11615_S16_rsem.genes.results,4mo,hipp,F,5xFADHEMI,TRUE
5,hipp_F_5xFADHEMI:Bin1HO_4mo_B2_13019_S32_rsem.genes.results,4mo,hipp,F,5xFADHEMI:Bin1HO,TRUE


In [105]:
unique(datTraits$Group)

[1] "5xFADHEMI"        "5xFADHEMI:Bin1HO" "5xFADWT"          "5xFADWT:Bin1HO"

In [106]:
## you should cahnge column name and keep variable based on your comparison you want to do
## if you need to do filtering in more than one columns you should copy and pase this part of the code
for (name in names(column_names)){
    for (i in c(1:dim(datTraits)[1])){
        if (!(datTraits[i, name] %in% column_names[[name]])){
            datTraits[i, "keep"] = FALSE
        }
    }
}

In [107]:
keep = datTraits[datTraits$keep, ]
datExpr_selected = datExpr[, keep$file.name]

### Define group

after you get your data it's time to define the group which is bassically categorize the parameter you want to investigate by doing DEG
here you need to define which parameter you want to investigte

In [108]:
parameter = "Group"
options = unique(keep[,parameter])
groups = rep(0, dim(datExpr_selected)[2])
for (i in c(1:dim(datExpr_selected)[2])) {
  if (keep[i,parameter] == options[1]){
    groups[i] = 1
  }
  if (keep[i,parameter] == options[2]){
    groups[i] = 2
  }
}
groups

[1] 1 1 1 1 2 2 2 2 2

## Running DEG using edgR

Now we have everythings, we only need to run it.

In [109]:
y = DGEList(counts = datExpr_selected, group=groups)
y = calcNormFactors(y)
design = model.matrix(~groups)
y = estimateDisp(y, design)

keep = filterByExpr(y)
y = y[keep, , keep.lib.sizes=FALSE]

#Testing for DE Genes
et = exactTest(y)

#extract table from the exact test( here is where we know if they are DE or not)
et_out = (topTags(et, n=Inf, adjust.method = "BH"))
et = et_out$table
et$gene_name = rownames(et)

In [110]:
#read gene list to convert gene ID to gene list
annot = read.table("genelist.vM21.annotation.tsv", sep="\t", header = T, row.name=1)

## convert gene ID to gene Name
for (i in c(1:dim(et)[1])){
    et$gene_name[i] = annot[et$gene_name[i], 'gene_name']
}

In [111]:
# label DE genes
# you can change therosholds base on your data!
et$DE = ""
et$DE[et$FDR < 0.05 & et$logFC > 0 ] = "Up"
et$DE[et$FDR < 0.05 & et$logFC < 0 ] = "Down"
et$DE[ et$DE == "" ] = "No"

table(et$DE)


   No    Up 
13923    44 

In [112]:
# save table
if (length(column_names[['Sex']]) == 1){
    name = paste0("DEG/table_DEG_", column_names[['Sex']], "_", column_names[['Time.point']], "_", column_names[['Group']][1], "_", column_names[['Group']][2], ".tsv")
} else {
    name = paste0("DEG/table_DEG_", column_names[['Time.point']], "_", column_names[['Group']][1], "_", column_names[['Group']][2], ".tsv")
}

write.table(et, name, sep="\t", row.names = F, quote = F)

## volcano plot

There is only one step left! plotting the results
if you change the default threshold you need to change it here as well!
For doing that `geom_vline` and `geom_hline` function

In [113]:
et$label = et$gene_name
for (i in c(1:dim(et)[1])) {
  if (et$DE[i] == "No") {
    et$label[i] = ""
  }
}

if (length(column_names[['Sex']]) == 1){
    name = paste0("DEG/plot_DEG_", column_names[['Sex']], "_", column_names[['Time.point']], "_", column_names[['Group']][1], "_", column_names[['Group']][2], ".pdf")
} else {
    name = paste0("DEG/plot_DEG_", column_names[['Time.point']], "_", column_names[['Group']][1], "_", column_names[['Group']][2], ".pdf")
}

color = c()
tmp = names(table(et$DE))
if ("Down" %in% tmp){
    color = c(color, "blue")
}
if ("No" %in% tmp){
    color = c(color, "black")
}
if ("Up" %in% tmp){
    color = c(color, "red")
}


pdf(name, width = 6, height = 6)
ggplot(data=et, aes(x=logFC, y=-log10(FDR), col=DE, label=label)) +
  geom_point() + 
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values=color) +
  geom_vline(xintercept=c(-0, 0), col="red") +
  geom_hline(yintercept=-log10(0.05), col="red")
dev.off();
dev.off();

Warning message:
"ggrepel: 35 unlabeled data points (too many overlaps). Consider increasing max.overlaps"


png 
  2

null device 
          1